In [1]:
#from gensim import utils
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
%matplotlib inline

In [2]:
train_data = pd.read_csv('../data/imdb_lm/train.csv', header=None, names=['label', 'text'])
test_data = pd.read_csv('../data/imdb_lm/test.csv', header=None, names=['label', 'text'])
train_texts = list(train_data.text.values)
train_labels = list(train_data.label.values)
test_texts = list(test_data.text.values)
test_labels = list(test_data.label.values)

In [3]:
train_data.head()

,label,text
0,0,I saw this movie when I attended a free screen...
1,0,I say: See this movie at least twice. First ti...
2,0,"This movie is so bad, and Jeffrey Comb's actin..."
3,0,Example of how a World War 2 documentary shoul...
4,0,Saw the premiere at GenArt festival in NYC on ...


## Clean Texts

In [3]:
#from tensorflow.contrib import learn

In [4]:
#MAX_DOCUMENT_LENGTH = 800
##vocab_processor = learn.preprocessing.VocabularyProcessor(MAX_DOCUMENT_LENGTH, min_frequency=2) # tensorflow提供的工具，将数据填充为最大长度，默认0填充
##train_sent_idx = np.array(list(vocab_processor.fit_transf
#                               orm(train_texts)))
#test_sent_idx = np.array(list(vocab_processor.transform(test_texts)))

In [5]:
#n_words = len(vocab_processor.vocabulary_)
#n_words

In [6]:
from tools.text_preprocess import text_clean
def clean_text(texts):
    '''
    Remove punctuation and lemmatize words
    '''
    tc = text_clean(texts, is_multiprocess=False)
    texts_processed = tc.proceed()
    return texts_processed

train_processed = clean_text(train_texts)
test_processed = clean_text(test_texts)

Start to process....
Processing Finished! Timing:  877.151
Start to process....
Processing Finished! Timing:  875.277


## Save the data

In [18]:
import pickle
with open('data/intermediate_data/training.pk', 'wb') as f:
    pickle.dump(train_processed, f, -1)
with open('data/intermediate_data/testing.pk', 'wb') as f:
    pickle.dump(test_processed, f, -1)

In [19]:
data = {'text': train_processed, 'sentiment':train_labels}
temp =  pd.DataFrame(data)
temp.to_csv('data/intermediate_data/train_processed.csv', index=False)
data = {'text': test_processed, 'sentiment':test_labels}
temp =  pd.DataFrame(data)
temp.to_csv('data/intermediate_data/test_processed.csv', index=False)

In [6]:
train_data = pd.read_csv('data/train_processed.csv')
test_data = pd.read_csv('data/test_processed.csv')
train_processed = list(train_data.text.values)
train_labels = list(train_data.sentiment.values)
test_processed = list(test_data.text.values)
test_labels = list(test_data.sentiment.values)

In [7]:
words = []
for item in train_processed:
    words.extend(item.split())

In [8]:
from collections import Counter
word_freq = Counter(words)

In [9]:
len(word_freq)

60008

## Map words to Ids

In [4]:
train_processed = train_texts
test_processed = test_texts

In [5]:
#Old one
#from tools.text_hier_split import sent2words
#from tools.token_idx_map import token2idx
#sw_train = sent2words(train_processed)
#sw_test = sent2words(test_processed)
#train_sent_words = list(sw_train.proceed())
#test_sent_words = list(sw_test.proceed())
#ti = token2idx(train_sent_words, 30000)
#train_sent_idx = ti.proceed()
#test_sent_idx = ti.map_text_idx(test_sent_words, ignore_sent=True)

In [6]:
from tools.text_hier_split import text2words
from tools.token_idx_map import token2idx
sw_train = text2words(train_processed)
sw_test = text2words(test_processed)
train_sent_words = sw_train.proceed()
test_sent_words = sw_test.proceed()
ti = token2idx(train_sent_words, 30000)
train_sent_idx = ti.proceed()
test_sent_idx = ti.map_text_idx(test_sent_words, ignore_sent=True)

Tokenization starting...
Tokenization Finished! Timing:  0.0
Tokenization starting...
Tokenization Finished! Timing:  0.0
Start mapping words to IDs....
Processing Finished! Timing:  29.411


In [9]:
n_words = len(ti.get_vocab())
print('Total words: %d' % n_words)

Total words: 30001


In [10]:
from tools.sample_generator import generate_samples
MAX_DOCUMENT_LENGTH = 800
gs_train = generate_samples(train_sent_idx, train_labels, MAX_DOCUMENT_LENGTH)
gs_test = generate_samples(test_sent_idx, test_labels, MAX_DOCUMENT_LENGTH)

## Settings

In [11]:
embed_size = 100
#MAX_DOCUMENT_LENGTH = 800
#n_words = 10000
class trainConfig:
    vocab_size = n_words
    max_doc_len = MAX_DOCUMENT_LENGTH
    label_size = 2
    embed_size = embed_size
    hidden_size = 250
    batch_size = 64
    layer_size = 2
    
class testConfig:
    vocab_size = n_words
    max_doc_len = MAX_DOCUMENT_LENGTH
    label_size = 2
    embed_size = embed_size
    hidden_size = 250
    batch_size = 64
    layer_size = 2
    
class singleConfig:
    vocab_size = n_words
    max_doc_len = MAX_DOCUMENT_LENGTH
    label_size = 2
    embed_size = embed_size
    hidden_size = 250#hidden size for hidden state of rnn
    batch_size = 1
    layer_size = 2

In [12]:
train_chunk_num = int(len(train_processed)/trainConfig.batch_size)
test_chunk_num = int(len(test_processed)/trainConfig.batch_size)
remain_num = len(test_processed) - trainConfig.batch_size*test_chunk_num
remain_num

16

## MultiChannelCNN

In [13]:
from models.CNN import CNN_Model

/Users/richardsun/anaconda3/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [14]:
import tensorflow as tf
graph_cnn = tf.Graph()
#Create models for training and testing data
#Create models for training and testing data
with graph_cnn.as_default():
    initializer = tf.random_uniform_initializer(-0.02, 0.02)
    with tf.name_scope('train'):
        #Set different models for different buckets
        with tf.variable_scope("Model", reuse=None, initializer=initializer):
            train_model = CNN_Model(trainConfig)
            saver=tf.train.Saver()
    with tf.name_scope('test'):
        #Set different models for different buckets
        with tf.variable_scope("Model", reuse=True, initializer=initializer):
            test_model = CNN_Model(testConfig, False)
            single_model = CNN_Model(singleConfig, False)

Model Initialized!
Model Initialized!
Model Initialized!


In [15]:
gs_train = generate_samples(train_sent_idx, train_labels, MAX_DOCUMENT_LENGTH)
gs_test = generate_samples(test_sent_idx, test_labels, MAX_DOCUMENT_LENGTH)

In [16]:
import time, os
epochs = 2
#train_chunk_num = 10
file = "ckpt_cnn/cnn.ckpt"
with tf.Session(graph=graph_cnn) as sess:
    #Initialize parameters
    init = tf.global_variables_initializer()
    if not os.path.exists("ckpt_cnn"):
        os.mkdir('ckpt_cnn')
    if os.path.exists("ckpt_cnn/cnn.ckpt.index"):
        saver.restore(sess, file)
    else:
         sess.run(init)
    start_time = time.time()
    for m in range(epochs):
        for i in range(train_chunk_num):
            #sess.run(tf.assign(learning_rate, 0.002*((0.98)**m)))
            x, y, lengths, _ = gs_train.generate_batch(trainConfig.batch_size)
            feed_dict = {train_model.x:x, train_model.y:y, train_model.lengths:lengths}
            l, _ = sess.run([train_model.cost, train_model.optimize], feed_dict=feed_dict)
            if i%100 == 0:
                print('Loss:', round(l, 4))
        end_time = time.time()
        print('Epoch', m, 'time:{:.2f}'.format(end_time - start_time))
        
    saver.save(sess,'ckpt_cnn/cnn.ckpt')

Loss: 0.6932


KeyboardInterrupt: 

In [28]:
#Calculate Testing Accuracy
with tf.Session(graph=graph_cnn) as sess:
    print('Testing...')
    count = 0
    #saver = tf.train.import_meta_graph('ckpt_cnn/cnn.ckpt.meta')
    saver.restore(sess,tf.train.latest_checkpoint('ckpt_cnn/'))
    print('Parameters restored')
    start_time = time.time()
    test_gs = generate_samples(np.array(test_processed), np.array(test_labels), False)
    for _ in range(test_chunk_num):
        #Traverse each data
        x, y, lengths, _ = gs_test.generate_batch(testConfig.batch_size)
        feed_dict = {test_model.x:x, test_model.y:y, test_model.lengths:lengths}
        n = sess.run(test_model.correct_num, feed_dict=feed_dict)
        count += np.sum(n)
    for _ in range(remain_num):
        #Traverse each data
        x, y, lengths, _ = gs_test.generate_batch(1)
        feed_dict = {single_model.x:x, single_model.y:y, 
                     single_model.lengths:lengths}
        n = sess.run(single_model.correct_num, feed_dict=feed_dict)
        count += np.sum(n)
    end_time = time.time()
    print('Testing Time:{:.2f}'.format(end_time - start_time))
    print(count*1.0/len(test_processed)) 

Testing...
INFO:tensorflow:Restoring parameters from ckpt_cnn/cnn.ckpt
Parameters restored
Testing Time:37.54
0.84472
